# Long Sentence Resolution - Evaluate Shortening Model

# Documentation

## Connect Google Drive

*   Run the cell "Connect Google Drive" and on the popup window click  "Connect to Google Drive".

*   In the new page opened, choose an account (your account) and click Allow.

## Configuration parameters

* **Source of test set generated predictions**: The source of test set generated predictions.

* **Source of test set ground-truth**: The source of test set ground-truth.

* **Target maximum #words**: The maximum number of words a sentence should have to be considered a valid target sentence (default: 16).

* **Minimum compression rate**: The minimum compression rate an example can have. All examples that have less than this threshold are dropped (default: 0).

# Installations

In [ ]:
!pip install evaluate==0.3.0    \
             rouge-score==0.1.2 \
             bert-score==0.3.11 \
             tqdm==4.64.1       \
             spacy==3.4.1

# Imports

In [ ]:
import os, evaluate

import numpy as np

import ipywidgets as widgets

import pandas as pd

from pathlib import Path

from typing import List, Union, Dict

from google.colab import drive

from spacy.lang.en import English

from spacy.tokens import Doc, Token

from tqdm import tqdm

# Functions / Classes

In [ ]:
class DocumentAnalysis:
  def __init__ (self):
    self._linguistics_model = English()

    self._linguistics_model.add_pipe("sentencizer")

  def extract_n_words_from_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> np.ndarray:
    return np.array(list(map(self._extract_n_words_from_document, self._linguistics_model.pipe(documents, batch_size=50))))

  def _extract_n_words_from_document(self, doc: Doc) -> int:
    return sum(map(self._token_is_word, doc))

  def _token_is_word(self, token: Token) -> bool:
    return not any([ token.is_bracket,
                     token.is_punct,
                     token.is_currency,
                     token.is_quote,
                     token.is_space ]) # https://spacy.io/api/token#attributes

class WordBasedCompressionRate:
  def __init__ (self):
    self._document_analysis = DocumentAnalysis()

  def compute(self, uncompressed_documents: Union[List[str], np.ndarray, pd.Series], compressed_documents: Union[List[str], np.ndarray, pd.Series], use_aggregator: bool = True) -> Union[Dict, float]:
    assert(len(uncompressed_documents) == len(compressed_documents))

    uncompressed_n_words = self._document_analysis.extract_n_words_from_documents(uncompressed_documents)

    compressed_n_words = self._document_analysis.extract_n_words_from_documents(compressed_documents)

    compression_rates = compressed_n_words / uncompressed_n_words

    return np.mean(compression_rates) if use_aggregator else {
        'compression_rates': compression_rates,
        'uncompressed_n_words': uncompressed_n_words,
        'compressed_n_words': compressed_n_words
    }

class ShorteningWordBasedAccuracy:
  def __init__ (self, n_words_threshold: int):
    self._document_analysis = DocumentAnalysis()

    self._n_words_threshold = n_words_threshold

  def compute(self, documents: Union[List[str], np.ndarray, pd.Series]) -> float:
    if len(documents) == 0: return 0

    return np.count_nonzero(self._document_analysis.extract_n_words_from_documents(documents) <= self._n_words_threshold) / len(documents)

class ShorteningEvaluationMetricsCalculation:
  def __init__ (self, target_maximum_n_words: int):
    self._bertscore_metric = evaluate.load("bertscore", module_type="metric")

    self._rouge_metric = evaluate.load('rouge', module_type="metric")

    self._word_based_accuracy_metric = ShorteningWordBasedAccuracy(n_words_threshold=target_maximum_n_words)

    self._word_based_compression_rate_metric = WordBasedCompressionRate()

  def _bert_score_f1(self, predictions: pd.Series, references: pd.Series, batch_size: int) -> float:
      batch_scores = []
      
      for i in tqdm(range(0, len(predictions), batch_size)):
          predictions_batch = predictions[i : i + batch_size]

          references_batch = references[i : i + batch_size]

          batch_f1 = self._bertscore_metric.compute(predictions=predictions_batch, references=references_batch, lang='en', batch_size=batch_size) ['f1']

          batch_scores.extend(batch_f1)

      return np.mean(batch_scores)

  def compute(self, predictions: pd.Series, sources: pd.Series, targets: pd.Series) -> pd.Series:
    predictions, sources, targets = predictions.values, sources.values, targets.values

    bert_score_f1_with_source = self._bert_score_f1(predictions=predictions, references=sources, batch_size=20)
    bert_score_f1_with_target = self._bert_score_f1(predictions=predictions, references=targets, batch_size=20)

    rouge_scores_with_source = self._rouge_metric.compute(predictions=predictions, references=sources, rouge_types=['rouge1', 'rouge2', 'rougeL'])
    rouge_scores_with_target = self._rouge_metric.compute(predictions=predictions, references=targets, rouge_types=['rouge1', 'rouge2', 'rougeL'])

    accuracy_percentage = 100.0 * self._word_based_accuracy_metric.compute(predictions)
    compression_rate_percentage = 100.0 * self._word_based_compression_rate_metric.compute(sources, predictions)

    return pd.Series([
        rouge_scores_with_source['rouge1'],
        rouge_scores_with_source['rouge2'],
        rouge_scores_with_source['rougeL'],

        rouge_scores_with_target['rouge1'],
        rouge_scores_with_target['rouge2'],
        rouge_scores_with_target['rougeL'],

        bert_score_f1_with_source,
        bert_score_f1_with_target,

        accuracy_percentage,
        compression_rate_percentage
    ], index=[ 'R1 (s)',
               'R2 (s)',
               'RL (s)',
               'R1 (t)',
               'R2 (t)',
               'RL (t)',
               'BERTScore F1 (s)',
               'BERTScore F1 (t)',
               'Accuracy (%)',
               'Compression Rate (%)' ])

# Connect Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Create paths

In [ ]:
MODELING_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/modeling')

MODELS_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Models')

# Configuration parameters

In [ ]:
test_set_generated_predictions_source_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('shortening_model'), os.listdir(MODELS_DIRECTORY_PATH))))
test_set_ground_truth_source_dropdown = widgets.Dropdown(options=list(filter(lambda o: o.startswith('shortening_dataset'), os.listdir(MODELING_DATA_DIRECTORY_PATH))))
target_maximum_n_words_bounded_int_text = widgets.BoundedIntText(value=16, min=1, max=20, step=1)
minimum_compression_rate_bounded_float_text = widgets.BoundedFloatText(value=0.0, min=0.0, max=1.0, step=0.05)

display(widgets.HBox([widgets.Label(value="Source of test set generated predictions:"), test_set_generated_predictions_source_dropdown]))
display(widgets.HBox([widgets.Label(value="Source of test set ground-truth:"), test_set_ground_truth_source_dropdown]))
display(widgets.HBox([widgets.Label(value="Target maximum #words:"), target_maximum_n_words_bounded_int_text]))
display(widgets.HBox([widgets.Label(value="Minimum compression rate:"), minimum_compression_rate_bounded_float_text]))

# Load test set generated predictions & ground-truth

In [ ]:
with open(MODELS_DIRECTORY_PATH / test_set_generated_predictions_source_dropdown.value / 'generated_predictions.txt', 'r') as f: predictions_df = pd.DataFrame(f.read().splitlines(), columns=['prediction'])

predictions_df

In [ ]:
test_set_df = pd.read_csv(MODELING_DATA_DIRECTORY_PATH / test_set_ground_truth_source_dropdown.value / 'test.csv', usecols=['dataset', 'source', 'target'])

test_set_df

In [ ]:
assert(len(test_set_df) == len(predictions_df))

In [ ]:
test_set_df['prediction'] = predictions_df['prediction']

test_set_df

# Measure compression rate

In [ ]:
word_based_compression_rate_metric = WordBasedCompressionRate()

In [ ]:
%%time

test_set_df['compression_rate'] = word_based_compression_rate_metric.compute(test_set_df['source'], test_set_df['target'], use_aggregator=False) ['compression_rates']

In [ ]:
test_set_df

# Filter source-target pairs

In [ ]:
test_set_df = test_set_df[test_set_df['compression_rate'] >= minimum_compression_rate_bounded_float_text.value]

In [ ]:
test_set_df

# Calculate metrics per dataset

In [ ]:
%%time

evaluation_metrics_calculation = ShorteningEvaluationMetricsCalculation(target_maximum_n_words=target_maximum_n_words_bounded_int_text.value)

test_set_df.groupby('dataset').apply(lambda o: evaluation_metrics_calculation.compute(o['prediction'], o['source'], o['target']))